In [2]:
import sqlite3
import pandas as pd

In [14]:
# Load the CSV file
data = pd.read_csv('/Users/ujwalnani/Documents/ELEVATE_ME/SQL_Capstone/HR-Analytics.csv')
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [8]:
# Connect to SQLite3 database
conn = sqlite3.connect('hr_analytics.db')
cursor = conn.cursor()


In [10]:
# Save the dataframe to the SQLite database
data.to_sql('employee_data', conn, if_exists='replace', index=False)
print("Database created and data imported successfully!")

Database created and data imported successfully!


In [16]:
# Verify the data in the database by printing the first 5 rows
query = "SELECT * FROM employee_data LIMIT 5;"
result = pd.read_sql(query, conn)
print("\nFirst 5 rows from the SQLite database:")
print(result)


First 5 rows from the SQLite database:
   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   ...  RelationshipSatisfaction S

In [22]:
# 1. Calculate Overall Attrition Rate
query = """
SELECT 
    CASE 
        WHEN Attrition = 'Yes' THEN 'true'
        ELSE 'false'
    END AS Attrition,
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM employee_data) AS Attrition_rate
FROM employee_data
GROUP BY Attrition;
"""
overall_attrition = pd.read_sql(query, conn)

# Format the Attrition_rate column to include percentage symbols
overall_attrition['Attrition_rate'] = overall_attrition['Attrition_rate'].map("{:.1f}%".format)

print(overall_attrition)

  Attrition Attrition_rate
0     false          83.9%
1      true          16.1%


In [26]:
# 2. Attrition by Gender

query = """
SELECT 
    CASE 
        WHEN Attrition = 'Yes' THEN 'true'
        ELSE 'false'
    END AS Attrition,
    Gender,
    COUNT(*) AS Count_gender,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Gender) AS Attrition_by_gender
FROM employee_data
GROUP BY Gender, Attrition;
"""

# Execute the query and load into a DataFrame
gender_attrition = pd.read_sql(query, conn)

# Format the Attrition_by_gender column to include percentage symbols
gender_attrition['Attrition_by_gender'] = gender_attrition['Attrition_by_gender'].map("{:.1f}%".format)

# Display the DataFrame
print(gender_attrition)


  Attrition  Gender  Count_gender Attrition_by_gender
0     false  Female           501               85.2%
1      true  Female            87               14.8%
2     false    Male           732               83.0%
3      true    Male           150               17.0%


In [28]:
# 3. Attrition by Department
query = """
SELECT 
    Department,
    Attrition,
    COUNT(*) AS Department_attrition
FROM employee_data
GROUP BY Department, Attrition;
"""

# Execute the query and load into a DataFrame
department_attrition = pd.read_sql(query, conn)

# Display the DataFrame
print(department_attrition)


               Department Attrition  Department_attrition
0         Human Resources        No                    51
1         Human Resources       Yes                    12
2  Research & Development        No                   828
3  Research & Development       Yes                   133
4                   Sales        No                   354
5                   Sales       Yes                    92


In [58]:
# 4. Attrition by Age Groups
query_age_groups = """
SELECT 
    CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 40 THEN '30 - 40'
        WHEN Age BETWEEN 40 AND 50 THEN '40 - 50'
        ELSE 'Over 50' 
    END AS Age_group,
    Attrition,
    COUNT(*) AS num,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY CASE 
        WHEN Age < 30 THEN 'Under 30'
        WHEN Age BETWEEN 30 AND 40 THEN '30 - 40'
        WHEN Age BETWEEN 40 AND 50 THEN '40 - 50'
        ELSE 'Over 50' 
    END), 2) AS percent_by_age
FROM 
    employee_data
GROUP BY 
    Age_group, Attrition
"""
attrition_by_age = pd.read_sql_query(query_age_groups, conn)
print("\nAttrition by Age Groups:")
print(attrition_by_age)



Attrition by Age Groups:
  Age_group Attrition  num  percent_by_age
0   30 - 40        No  585           86.16
1   30 - 40       Yes   94           13.84
2   40 - 50        No  288           89.44
3   40 - 50       Yes   34           10.56
4   Over 50        No  125           87.41
5   Over 50       Yes   18           12.59
6  Under 30        No  235           72.09
7  Under 30       Yes   91           27.91


In [60]:
#5. Attrition by Monthly Income and Job Level
query_income_job_level = """
SELECT 
    Department,
    JobLevel,
    ROUND(AVG(MonthlyIncome), 1) AS avg_income,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome ELSE NULL END), 1) AS attrition_avg_income,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome ELSE NULL END) - AVG(MonthlyIncome), 1) AS difference
FROM 
    employee_data
GROUP BY 
    Department, JobLevel
"""
attrition_by_income_job = pd.read_sql_query(query_income_job_level, conn)
print("\nAttrition by Department, Job Level, and Monthly Income:")
print(attrition_by_income_job)



Attrition by Department, Job Level, and Monthly Income:
                Department  JobLevel  avg_income  attrition_avg_income  \
0          Human Resources         1      2733.2                2415.7   
1          Human Resources         2      5563.5                   NaN   
2          Human Resources         3      9623.0               10216.0   
3          Human Resources         4     16147.5                   NaN   
4          Human Resources         5     19197.9                   NaN   
5   Research & Development         1      2840.1                2687.4   
6   Research & Development         2      5291.2                5372.0   
7   Research & Development         3     10170.5                9503.8   
8   Research & Development         4     15634.7               12169.0   
9   Research & Development         5     19218.5               19550.0   
10                   Sales         1      2506.7                2373.4   
11                   Sales         2      5746.1       

In [66]:
#6. Find Attrition by Years At Company
# Query to group employees into tenure ranges and calculate attrition
query_tenure_groups = """
SELECT 
    CASE 
        WHEN YearsAtCompany BETWEEN 0 AND 1 THEN 'New Hires'
        WHEN YearsAtCompany BETWEEN 2 AND 5 THEN '2-5 years'
        WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6-10 years'
        WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11-20 years'
        ELSE 'Over 20 years'
    END AS TenureYears,
    COUNT(*) AS TotalEmployees,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS AttritionCount,
    ROUND((SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS AttritionRate
FROM 
    employee_data
GROUP BY 
    TenureYears
ORDER BY 
    CASE 
        WHEN TenureYears = 'New Hires' THEN 1
        WHEN TenureYears = '2-5 years' THEN 2
        WHEN TenureYears = '6-10 years' THEN 3
        WHEN TenureYears = '11-20 years' THEN 4
        ELSE 5
    END
"""

# Execute the query
tenure_attrition = pd.read_sql_query(query_tenure_groups, conn)

# Rename columns for clarity and better formatting
tenure_attrition.rename(
    columns={
        "TenureYears": "tenure_years",
        "AttritionCount": "num",
        "AttritionRate": "percent",
    },
    inplace=True,
)

print(tenure_attrition.to_string(index=False))


 tenure_years  TotalEmployees  num  percent
    New Hires             215   75    34.88
    2-5 years             561   87    15.51
   6-10 years             448   55    12.28
  11-20 years             180   12     6.67
Over 20 years              66    8    12.12


In [68]:
##1. Why do more people over 50 years old leave the company than people aged 40–50?

query_attrition_over_50 = """
SELECT 
    CASE 
        WHEN Age BETWEEN 40 AND 50 THEN '40-50'
        WHEN Age > 50 THEN 'Over 50'
    END AS AgeGroup,
    COUNT(*) AS TotalEmployees,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS AttritionCount,
    ROUND((SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS AttritionRate
FROM 
    employee_data
WHERE 
    Age > 40
GROUP BY 
    AgeGroup
"""
attrition_over_50 = pd.read_sql_query(query_attrition_over_50, conn)
print("\nAttrition by Age Group (40-50 and Over 50):")
print(attrition_over_50)


Attrition by Age Group (40-50 and Over 50):
  AgeGroup  TotalEmployees  AttritionCount  AttritionRate
0    40-50             322              34          10.56
1  Over 50             143              18          12.59


## Why More People Over 50 Leave?
This query compares employees over 50 with those aged 40-50. Possible reasons might include retirement plans, reduced adaptability to company changes, or other personal factors.

In [72]:
##2. Why do people with higher pay still leave the company?

query_high_pay_attrition = """
SELECT 
    JobLevel,
    ROUND(AVG(MonthlyIncome), 2) AS AverageIncome,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS AttritionCount,
    ROUND((SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS AttritionRate
FROM 
    employee_data
GROUP BY 
    JobLevel
ORDER BY 
    AverageIncome DESC
"""
high_pay_attrition = pd.read_sql_query(query_high_pay_attrition, conn)
print("\nAttrition for Employees with Higher Pay (Grouped by Job Level):")
print(high_pay_attrition)


Attrition for Employees with Higher Pay (Grouped by Job Level):
   JobLevel  AverageIncome  AttritionCount  AttritionRate
0         5       19191.83               5           7.25
1         4       15503.78               5           4.72
2         3        9817.25              32          14.68
3         2        5502.28              52           9.74
4         1        2786.92             143          26.34


## High Pay and Attrition
The query shows attrition rates across job levels. People with higher pay may leave due to work-life balance, job satisfaction, or better opportunities elsewhere.

In [77]:
## 3. Which factors drive employees who work at the company less than 5 years to leave?

query_attrition_under_5_years = """
SELECT 
    YearsAtCompany,
    COUNT(*) AS TotalEmployees,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS AttritionCount,
    ROUND((SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS AttritionRate,
    ROUND(AVG(DistanceFromHome), 2) AS AvgDistanceFromHome,
    ROUND(AVG(WorkLifeBalance), 2) AS AvgWorkLifeBalance,
    ROUND(AVG(JobSatisfaction), 2) AS AvgJobSatisfaction,
    ROUND(AVG(EnvironmentSatisfaction), 2) AS AvgEnvironmentSatisfaction
FROM 
    employee_data
WHERE 
    YearsAtCompany < 5
GROUP BY 
    YearsAtCompany
ORDER BY 
    YearsAtCompany
"""
attrition_under_5_years = pd.read_sql_query(query_attrition_under_5_years, conn)
print("\nFactors Driving Attrition for Employees with Less than 5 Years at the Company:")
print(attrition_under_5_years)


Factors Driving Attrition for Employees with Less than 5 Years at the Company:
   YearsAtCompany  TotalEmployees  AttritionCount  AttritionRate  \
0               0              44              16          36.36   
1               1             171              59          34.50   
2               2             127              27          21.26   
3               3             128              20          15.63   
4               4             110              19          17.27   

   AvgDistanceFromHome  AvgWorkLifeBalance  AvgJobSatisfaction  \
0                 8.48                2.75                2.59   
1                 9.43                2.77                2.72   
2                 8.87                2.73                2.87   
3                 8.05                2.80                2.59   
4                 9.06                2.63                2.60   

   AvgEnvironmentSatisfaction  
0                        2.80  
1                        2.69  
2                 

## Factors Driving Attrition for Less than 5 Years
The last query identifies factors such as distance from home, work-life balance, job satisfaction, and environmental satisfaction for new employees (less than 5 years) who left.